<a href="https://colab.research.google.com/github/kridtapon/5-volatility-indicator/blob/main/5_volatility_indicator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import plotly.graph_objects as go

# Fetch the S&P 500 data (you can change the ticker if needed)
ticker = "^GSPC"  # S&P 500 index symbol
data = yf.download(ticker, start="2022-01-01", end="2024-01-01")
data.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
data.ffill(inplace=True)

# Create a candlestick chart
fig = go.Figure(data=[go.Candlestick(
    x=data.index,
    open=data['Open'],
    high=data['High'],
    low=data['Low'],
    close=data['Close'],
    name="Candlesticks"
)])

# Update layout for better visual clarity
fig.update_layout(
    title="S&P 500 Candlestick Chart",
    xaxis_title="Date",
    yaxis_title="Price",
    xaxis_rangeslider_visible=False,
    template="plotly_dark"
)

# Show the interactive chart
fig.show()


[*********************100%***********************]  1 of 1 completed


In [ ]:
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Fetch the S&P 500 data (you can change the ticker if needed)
ticker = "^GSPC"  # S&P 500 index symbol
data = yf.download(ticker, start="2022-01-01", end="2024-01-01")
data.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
data.ffill(inplace=True)

# Define the Chaikin Volatility function
def chaikin_volatility(data, period=10):
    data['HL'] = data['High'] - data['Low']
    data['CHV'] = data['HL'].ewm(span=period, adjust=False).mean()
    data['ChaikinVolatility'] = data['CHV'].pct_change(periods=period) * 100
    return data

# Apply the Chaikin Volatility function to the data
data = chaikin_volatility(data)

# Create the subplot figure
fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.3)

# Plot the Chaikin Volatility indicator
fig.add_trace(go.Scatter(x=data.index, y=data['CHV'], mode='lines', name='Chaikin Volatility', line=dict(color='orange')), row=1, col=1)

# Update layout for better presentation
fig.update_layout(
    title="Chaikin Volatility Indicator (S&P 500)",
    xaxis_title="Date",
    yaxis_title="Chaikin Volatility",
    template="plotly_dark"
)

# Show the plot
fig.show()


[*********************100%***********************]  1 of 1 completed


In [ ]:
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Fetch the S&P 500 data (you can change the ticker if needed)
ticker = "^GSPC"  # S&P 500 index symbol
data = yf.download(ticker, start="2022-01-01", end="2024-01-01")
data.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
data.ffill(inplace=True)

# Define the Donchian Channels function
def donchian_channels(data, period=20):
    data['Upper'] = data['High'].rolling(window=period).max()
    data['Lower'] = data['Low'].rolling(window=period).min()
    data['Middle'] = (data['Upper'] + data['Lower']) / 2
    return data

# Apply the Donchian Channels function to the data
data = donchian_channels(data)

# Create the subplot figure
fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.3,
                    subplot_titles=["Donchian Channels with Candlestick"],
                    row_heights=[0.7],
                    row_width=[0.3])

# Plot the Candlestick chart
fig.add_trace(go.Candlestick(x=data.index,
                             open=data['Open'],
                             high=data['High'],
                             low=data['Low'],
                             close=data['Close'],
                             name="Candlestick",
                             increasing_line_color='green',
                             decreasing_line_color='red'), row=1, col=1)

# Plot the Donchian Channels
fig.add_trace(go.Scatter(x=data.index, y=data['Upper'], mode='lines', name='Upper Channel', line=dict(color='green')), row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['Lower'], mode='lines', name='Lower Channel', line=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['Middle'], mode='lines', name='Middle Channel', line=dict(color='blue', dash='dot')), row=1, col=1)

# Update layout for better presentation
fig.update_layout(
    title="Donchian Channels with Candlestick (S&P 500)",
    xaxis_title="Date",
    yaxis_title="Price",
    template="plotly_dark",
    xaxis_rangeslider_visible=False
)

# Show the plot
fig.show()


[*********************100%***********************]  1 of 1 completed


In [ ]:
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define the Keltner Channels function
def keltner_channels(data, period=20, atr_multiplier=2):
    data['TR'] = data[['High', 'Low', 'Close']].apply(
        lambda x: max(
            x['High'] - x['Low'],
            abs(x['High'] - x['Close']),
            abs(x['Low'] - x['Close'])
        ), axis=1
    )
    data['ATR'] = data['TR'].rolling(window=period).mean()
    data['Middle'] = data['Close'].rolling(window=period).mean()
    data['Upper'] = data['Middle'] + atr_multiplier * data['ATR']
    data['Lower'] = data['Middle'] - atr_multiplier * data['ATR']
    return data

# Fetch the S&P 500 data (you can change the ticker if needed)
ticker = "^GSPC"  # S&P 500 index symbol
data = yf.download(ticker, start="2022-01-01", end="2024-01-01")
data.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
data.ffill(inplace=True)

# Apply the Keltner Channels function to the data
data = keltner_channels(data)

# Create the subplot figure
fig = make_subplots(
    rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.3,
    subplot_titles=["Keltner Channels with Candlestick"],
    row_heights=[0.7],
    row_width=[0.3]
)

# Plot the Candlestick chart
fig.add_trace(go.Candlestick(
    x=data.index,
    open=data['Open'],
    high=data['High'],
    low=data['Low'],
    close=data['Close'],
    name="Candlestick",
    increasing_line_color='green',
    decreasing_line_color='red'
    ), row=1, col=1)

# Plot the Keltner Channels
fig.add_trace(go.Scatter(x=data.index, y=data['Upper'], mode='lines', name='Upper Channel', line=dict(color='green')), row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['Lower'], mode='lines', name='Lower Channel', line=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['Middle'], mode='lines', name='Middle Channel', line=dict(color='blue', dash='dot')), row=1, col=1)

# Update layout for better presentation
fig.update_layout(
    title="Keltner Channels with Candlestick (S&P 500)",
    xaxis_title="Date",
    yaxis_title="Price",
    template="plotly_dark",
    xaxis_rangeslider_visible=False
)

# Show the plot
fig.show()


[*********************100%***********************]  1 of 1 completed


In [ ]:
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Fetch the S&P 500 data (you can change the ticker if needed)
ticker = "^GSPC"  # S&P 500 index symbol
data = yf.download(ticker, start="2022-01-01", end="2024-01-01")
data.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
data.ffill(inplace=True)

# Define the Relative Volatility Index (RVI) function
def relative_volatility_index(data, period=14):
    # Calculate the difference between the Close and Open prices
    data['Upward Volatility'] = np.where(data['Close'] > data['Open'], data['Close'] - data['Open'], 0)
    data['Total Volatility'] = abs(data['Close'] - data['Open'])

    # Calculate the rolling sum of Upward Volatility and Total Volatility
    data['Upward Volatility Sum'] = data['Upward Volatility'].rolling(window=period).sum()
    data['Total Volatility Sum'] = data['Total Volatility'].rolling(window=period).sum()

    # Calculate RVI as the ratio of Upward Volatility to Total Volatility
    data['RVI'] = data['Upward Volatility Sum'] / data['Total Volatility Sum'] * 100

    return data

# Apply the Relative Volatility Index (RVI) function to the data
data = relative_volatility_index(data)

# Create the subplot figure
fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.3)

# Plot the RVI indicator
fig.add_trace(go.Scatter(x=data.index, y=data['RVI'], mode='lines', name='Relative Volatility Index', line=dict(color='orange')), row=1, col=1)

# Update layout for better presentation
fig.update_layout(
    title="Relative Volatility Index (RVI) Indicator (S&P 500)",
    xaxis_title="Date",
    yaxis_title="RVI Value",
    template="plotly_dark"
)

# Show the plot
fig.show()




[*********************100%***********************]  1 of 1 completed


In [ ]:
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Fetch the S&P 500 data (you can change the ticker if needed)
ticker = "^GSPC"  # S&P 500 index symbol
data = yf.download(ticker, start="2022-01-01", end="2024-01-01")
data.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
data.ffill(inplace=True)

# Define the standard deviation function
def standard_deviation(data, period=20):
    data['Std_Dev'] = data['Close'].rolling(window=period).std()
    return data

# Apply the standard deviation function to the data
data = standard_deviation(data)

# Create the subplot figure
fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.3)

# Plot the standard deviation indicator
fig.add_trace(go.Scatter(x=data.index, y=data['Std_Dev'], mode='lines', name='Standard Deviation', line=dict(color='orange')), row=1, col=1)

# Update layout for better presentation
fig.update_layout(
    title="Standard Deviation Indicator (S&P 500)",
    xaxis_title="Date",
    yaxis_title="Standard Deviation",
    template="plotly_dark"
)

# Show the plot
fig.show()


[*********************100%***********************]  1 of 1 completed
